In [1]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

In [3]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(r"C:\Users\user\Downloads\spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [4]:
# Create a dash application
app = dash.Dash(__name__)

In [14]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
# TASK 1: Add a dropdown list to enable Launch Site selection
# The default select value is for ALL sites
# dcc.Dropdown(id='site-dropdown',...)
html.Br(),
dcc.Dropdown(id='site_dropdown',
            options=[{'label': 'All Sites', 'value': 'ALL'},
                    {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'},
                    {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'},
                    {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}
                    ],
            value='ALL', placeholder='Do select a site',searchable=True),
html.Br(),
# TASK 2: Add a pie chart to show the total successful launches count for all sites
# If a specific launch site was selected, show the Success vs. Failed counts for the site
html.Div(dcc.Graph(id='success-pie-chart')),
html.Br(),
                                
                               ])

In [15]:
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site_dropdown', component_property='value'))
def get_pie_chart(site_dropdown):
    filtered_df = spacex_df[spacex_df['class'] == 1]
    if site_dropdown == 'ALL':
        fig = px.pie(data, values = 'class', names='Launch Site', title='Successful Lanch')
    #return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site']== site_dropdown]
        fig = px.pie(spacex_df, names = 'class',
        title = 'Successful Launch')
    return fig

In [19]:
# Run the app
if __name__ == '__main__':
    app.run_server(mode='inline')

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


TypeError: run_simple() got an unexpected keyword argument 'mode'

In [ ]:
# Import required libraries
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

# Read the airline data into pandas dataframe
spacex_df = pd.read_csv(r'C:\Users\user\Downloads\spacex_launch_dash.csv')
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',  'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown (id='site-dropdown', options=[
                                                {'label': 'All Sites', 'value': 'ALL'},
                                                {'label': 'CCAFS LC-40', 'value': 'CCAFS LC-40'}, 
                                                {'label': 'VAFB SLC-4E', 'value': 'VAFB SLC-4E'}, 
                                                {'label': 'KSC LC-39A', 'value': 'KSC LC-39A'}, 
                                                {'label': 'CCAF SLC-40', 'value': 'CCAFS SLC-40'}],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider', min=0, max=10000, step=1000,
                                        marks={0: '0', 100: '100'}, value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))

def update_pie (entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        pie_fig = px.pie(filtered_df, values='name size', names='Launch Site', 
                        title=f'Total Success Launches by Site')
        return pie_fig
    else:
        filtered_dfs = filtered_df[filtered_df['Launch Site'] == entered_site].groupby('class').size().reset_index(name='class count')
        pie_fig = px.pie(filtered_dfs, values='class size', names='class', 
                       title=f'Total Success Launches for Site {entered_site}')
        return pie_fig

# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
                [Input(component_id='site-dropdown', component_property='value'), 
                Input(component_id='payload-slider', component_property='value')])

def update_scatter (entered_site, slider_position):
    low, high = slider_position
    scat_df = spacex_df[spacex_df['Payload Mass (kg)'].between(low, high)]
    if entered_site == 'ALL':
        scat_fig = px.scatter(scat_df, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                         title=f'Correlation between Payload and Success for All Sites') 
        return scat_fig
    else:
        scat_dfs = scat_df[scat_df['Launch Site'] == entered_site]
        scat_fig = px.scatter(scat_dfs, x="Payload Mass (kg)", y="class", color="Booster Version Category",
                         title=f'Correlation between Payload and Success for Site {entered_site}') 
        return scat_fig

# Run the app
if __name__ == '__main__':
    app.run_server(debug=False) 

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Feb/2022 19:30:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 19:30:15] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_9_1m1632330794.8.7.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 19:30:16] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_3m1632330797.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 19:30:16] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_16_0m1632330795.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 19:30:16] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_9_1m1632330794.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 19:30:16] "GET /_dash-component-suites/dash_renderer/react@16.v1_9_1m1632330794.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 19:30:16] "GET /_dash-component-suites/dash_core_components/dash_core_components.v1_16_0m1632330795.min.js HTTP/1.1" 200 -
127.0.0.1 - - [02/

Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "C:\Users\user\anaconda3a\lib\site-packages\flask\app.py", line 2446, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\Users\user\anaconda3a\lib\site-packages\flask\app.py", line 1951, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\Users\user\anaconda3a\lib\site-packages\flask\app.py", line 1820, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\Users\user\anaconda3a\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\Users\user\anaconda3a\lib\site-packages\flask\app.py", line 1949, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\user\anaconda3a\lib\site-packages\flask\app.py", line 1935, in dispatch_request
    return self.view_functions[rule.endpoint](**req.view_args)
  File "C:\Users\user\anaconda3a\lib\site-packages\dash\dash.py", line 1079, in dispatch
    response.set_d

127.0.0.1 - - [02/Feb/2022 19:30:20] "POST /_dash-update-component HTTP/1.1" 500 -
127.0.0.1 - - [02/Feb/2022 19:30:45] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Feb/2022 19:30:45] "POST /_dash-update-component HTTP/1.1" 200 -
